# Ikeda for many ships
The method developed in: ([01.03_ikeda_many_dev](06_ikeda/01.03_ikeda_many_dev.ipynb)) will now be attempted for many ships.

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile
from pyscores2 import TDPError
import pyscores2
from rolldecayestimators.ikeda import Ikeda, IkedaR
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda
import subprocess

In [ ]:
df_all_sections_id = pd.read_csv('all_sections.csv', sep=';')
df_all_sections_id.head()

In [ ]:
section_groups=df_all_sections_id.groupby(by='loading_condition_id')

In [ ]:
loading_condition_ids = df_all_sections_id['loading_condition_id'].unique()
mask=pd.notnull(loading_condition_ids)
loading_condition_ids=loading_condition_ids[mask]

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
mask=df_rolldecay['loading_condition_id'].isin(loading_condition_ids)
df=df_rolldecay.loc[mask].copy()

In [ ]:
def add_cScores(sections):
    sections=sections.copy()
    sections['cScores']=sections['area']/(sections['b']*sections['t'])
    mask=sections['cScores']>1
    sections.loc[mask,'cScores']=1
    return sections

def cut_sections(sections, draught):
    sections=sections.copy()
    mask = sections['t']>draught
    sections.loc[mask,'t']=draught
    sections.loc[mask,'area']-=draught*sections['b'].max()  # Assuming rectangular shape
    return sections

def remove_duplicate_sections(sections):
    sections=sections.copy()
    mask=~sections['x'].duplicated()
    sections=sections.loc[mask]
    assert sections['x'].is_unique
    return sections

def too_small_sections(sections):
    sections=sections.copy()
    small = 0.1
    mask=sections['b']==0
    sections.loc[mask,'b']=small
    mask=sections['t']==0
    sections.loc[mask,'t']=small
    mask=sections['area']==0
    sections.loc[mask,'area']=small
    return sections

In [ ]:
from scipy.integrate import simps
def calculate_lcb(x, area, **kwargs):
    """
    Calculate lcb from AP
    """
    return simps(y=area*x,x=x)/np.trapz(y=area,x=x)

def calculate_dispacement(x, area, **kwargs):
    """
    Calculate displacement
    """
    return np.trapz(y=area,x=x)

In [ ]:
class DraughtError(ValueError): pass

def define_indata(row, sections, rho=1000,  g=9.81):
    
    indata = Indata()
    
    draught=(row.TA+row.TF)/2
    indata.draught=draught
    if draught<=sections['t'].max():
        sections = cut_sections(sections, draught)
    else:
        raise DraughtError('Draught is too large for sections')
    
    sections=add_cScores(sections)
    
    indata.cScores=np.array(sections['cScores'])
    indata.ts=np.array(sections['t'])
    indata.bs=np.array(sections['b'])
    indata.zbars=np.zeros_like(sections['b'])  # Guessing...
        
    beam=sections['b'].max()
    indata.lpp=sections['x'].max()-sections['x'].min()
    #indata.displacement=row.Volume
    indata.displacement=calculate_dispacement(**sections)
    
    indata.g=g
    indata.kxx=row.KXX
    indata.kyy=row.lpp*0.4
    lcb=calculate_lcb(x=sections['x'], area=sections['area'])
    indata.lcb=lcb-row.lpp/2
    indata.lpp=row.lpp
    indata.projectName='loading_condition_id_%i' % row.loading_condition_id
    
    indata.rho=rho
    indata.zcg=row.kg-draught
    #indata.waveFrequenciesMin=0.2
    #indata.waveFrequenciesMax=0.5
    #indata.waveFrequenciesIncrement=0.006
    w=row.omega0/np.sqrt(row.scale_factor)
    indata.waveFrequenciesMin=w*0.5
    indata.waveFrequenciesMax=w*2.0
    N=40
    indata.waveFrequenciesIncrement=(indata.waveFrequenciesMax-indata.waveFrequenciesMin)/N
    indata.runOptions["IE"].set_value(1)
    
    return indata,sections

In [ ]:
def create_ikeda(row, indata, output_file, fi_a):

    w = row.omega0
    scale_factor=row.scale_factor
    V = row.ship_speed*1.852/3.6/np.sqrt(scale_factor)
    R = 0.01*row.beam/scale_factor
    lBK=row.BKL/scale_factor
    bBK=row.BKB/scale_factor
    ikeda = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                                scale_factor=scale_factor, lBK=lBK, bBK=bBK)
    
    ikeda.R = R
    return ikeda

In [ ]:
def calculate_ikeda(ikeda):

    output = {}
    output['B_44_hat'] = ikeda.calculate_B44()[0]
    output['B_W0_hat'] =ikeda.calculate_B_W0()[0]
    output['B_W_hat'] =ikeda.calculate_B_W()[0]
    output['B_F_hat'] =ikeda.calculate_B_F()[0]
    output['B_E_hat'] =ikeda.calculate_B_E()[0]
    output['B_BK_hat'] =ikeda.calculate_B_BK()[0]
    output['B_L_hat'] =ikeda.calculate_B_L()[0]
    output['Bw_div_Bw0'] =ikeda.calculate_Bw_div_Bw0()[0]
    return output

In [ ]:
results = pd.DataFrame()
fi_a = np.deg2rad(10)
for run_name, row in df.iterrows():
    loading_condition_id=row['loading_condition_id']
    sections = section_groups.get_group(loading_condition_id)
    sections=remove_duplicate_sections(sections)
    sections=too_small_sections(sections)
    
    try:
        indata,sections_ = define_indata(row, sections)
    except DraughtError as e:
        print('Draught is too large for sections, this loading condition is skipped.')
        continue

        
    save_name='%s.in' % row.loading_condition_id
    save_path=os.path.join('scores2',save_name)
    indata.save(save_path)
    
    calculation = Calculation(outDataDirectory='scores2/result')
    
    # Run scoresII:
    try:
        calculation.run(indata=indata, b_div_t_max=None, timeout=1.0)
    except TDPError:
        print('Dissregarding the TDPError')
        continue
    except pyscores2.LcgError as e:
        print('Disregarded')
        print(e)
        continue
    except subprocess.TimeoutExpired:
        print('Disregarded, scoresII got stuck...')
        continue
        
    output_file = OutputFile(filePath=calculation.outDataPath)
    ikeda = create_ikeda(row=row, indata=indata, output_file=output_file, fi_a=fi_a)
    result_data = calculate_ikeda(ikeda)
    result=pd.Series(data=result_data, name=row.name)
    results=results.append(result)
    

In [ ]:
results

## Also run Simplified Ikeda for comparison

In [ ]:
def calculate_si(si):

    output = pd.DataFrame()
    output['B_44_hat'] = si.calculate_B44()
    output['B_W0_hat'] =si.calculate_B_W0()
    output['B_W_hat'] =si.calculate_B_W()
    output['B_F_hat'] =si.calculate_B_F()
    output['B_E_hat'] =si.calculate_B_E()
    output['B_BK_hat'] =si.calculate_B_BK()
    output['B_L_hat'] =si.calculate_B_L()
    output['Bw_div_Bw0'] =si.calculate_Bw_div_Bw0()
    return output

In [ ]:
inputs_si=pd.DataFrame()
inputs_si['w']=df['omega0']  # Already model scale
scale_factor=df['scale_factor']
inputs_si['V']=df['ship_speed']*1.852/3.6/np.sqrt(scale_factor)
inputs_si['fi_a']=fi_a
inputs_si['beam']=df['beam']/scale_factor
inputs_si['lpp']=df['lpp']/scale_factor
inputs_si['kg']=df['kg']/scale_factor
inputs_si['volume']=df['Volume']/(scale_factor**3)
draught=(df['TA']+df['TF'])/2
inputs_si['draught']=draught/scale_factor
inputs_si['A0']=df['A0']
inputs_si['lBK']=df['BKL']/scale_factor
inputs_si['bBK']=df['BKB']/scale_factor
si = SimplifiedIkeda(**inputs_si)
results_si = calculate_si(si)
results_si.index=df.index

## Make comparison with model tests

In [ ]:
B_e = lambdas.B_e_lambda(B_1=df['B_1'], B_2=df['B_2'], phi_a=fi_a, 
                   omega0=df['omega0'])

scale_factor = df['scale_factor']
Volume = df['Volume']/(scale_factor**3)
beam = df['beam']/scale_factor
g=9.81
rho=1000

df['B_e_hat'] = lambdas.B_e_hat_lambda(B_e=B_e, Disp=Volume, beam=beam, 
                                 g=g, rho=rho)

In [ ]:
df_results = pd.merge(left=results, right=results_si, how='inner', left_index=True, right_index=True,
                     suffixes=('_ikeda','_si'))

In [ ]:
mask = df_results['B_44_hat_ikeda'].notnull()
df_results = df_results.loc[mask].copy()


df_compare = pd.merge(left=df, right=df_results, how='inner', left_index=True, right_index=True,
                     suffixes=('','_y'))


In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_44_hat_ikeda', y='B_44_hat_si', ax=ax, style='o')

ax.set_xlabel(r'$\hat{B_{44}}$ (Ikeda)')
ax.set_ylabel(r'$\hat{B_{44}}$ (SI)')

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')

ax.grid(True)
ax.set_aspect('equal', 'box')

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_e_hat', y='B_44_hat_ikeda', ax=ax, style='.', 
                label=r'$\hat{B_{44}}$ Ikeda')

df_compare.plot(x='B_e_hat', y='B_44_hat_si', ax=ax, style='.', 
                label=r'$\hat{B_{44}}$ SI')

ax.set_xlabel(r'$\hat{B_{44}}$ (model test)')
ax.set_ylabel(r'$\hat{B_{44}}$ (prediction)')

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.set_title('Total roll damping for Ikeda, Simplified Ikeda and model tests')
ax.plot([0,lim],[0,lim],'r-')

ax.grid(True)
ax.set_aspect('equal', 'box')
ax.legend();

In [ ]:
r2_score(y_true=df_compare['B_e_hat'], y_pred=df_compare['B_44_hat_ikeda'])

In [ ]:
r2_score(y_true=df_compare['B_e_hat'], y_pred=df_compare['B_44_hat_si'])

## Investigating the residuals
<a id="residuals"></a>

In [ ]:
def calculate_residuals(suffix_true='_ikeda', suffix_prediction='_si'):
    prefixes = ['B_44_hat',
                'B_W0_hat',
                'B_W_hat',
                'B_F_hat',
                'B_E_hat',
                'B_BK_hat',
                'B_L_hat', 
                'Bw_div_Bw0',]
    
    for prefix in prefixes:
        residual_name = '%s_residual%s%s' % (prefix, suffix_prediction, suffix_true)
        name_true='%s%s' % (prefix, suffix_true)
        name_prediction='%s%s' % (prefix, suffix_prediction)
        
        df_compare[residual_name] = df_compare[name_prediction] - df_compare[name_true]



In [ ]:
calculate_residuals()

In [ ]:
import seaborn as sns; 
#sns.set_theme()

In [ ]:
df_compare['draught']=(df_compare['TA'] + df_compare['TF'])/2
df_compare[r'LBK/Lpp']=df_compare['BKL']/df_compare['lpp']
df_compare[r'BBK/beam']=df_compare['BKB']/df_compare['beam']
df_compare['omega_hat']=lambdas.omega_hat(beam=df_compare['beam'], g=g, omega0=df_compare['omega0'])

df_compare['Cb']=df_compare['Volume']/(df_compare['lpp']*df_compare['beam']*df_compare['draught'])
interesting=['Cb','A0','OG/d','LBK/Lpp','BBK/beam','omega_hat',r'beam/draught', 'Fn']
sns.pairplot(df_compare,y_vars='B_44_hat_residual_si_ikeda', x_vars=interesting, aspect=0.6);

In [ ]:
interesting2=[
#'B_44_hat_residual_si_ikeda',
'B_W0_hat_residual_si_ikeda',
'B_W_hat_residual_si_ikeda',
'B_F_hat_residual_si_ikeda',
'B_E_hat_residual_si_ikeda',
'B_BK_hat_residual_si_ikeda',
'B_L_hat_residual_si_ikeda', 
]

sns.pairplot(df_compare,y_vars='residual_si', x_vars=interesting2, aspect=0.6);

In [ ]:
suffix_true='_ikeda'
suffix_prediction='_si'
prefixes = [
    'B_W0_hat',
    'B_W_hat',
    'B_F_hat',
    'B_E_hat',
    'B_BK_hat',
    'B_L_hat',]

lim = np.max([df_compare['B_44_hat_ikeda'].max(),
              df_compare['B_44_hat_si'].max(),
             ])
    

for prefix in prefixes:
    
    name_true='%s%s' % (prefix, suffix_true)
    name_prediction='%s%s' % (prefix, suffix_prediction)
        
    fig,ax=plt.subplots()
    df_compare.plot(x=name_true, y=name_prediction, ax=ax, style='.')
    
    
    #ax.set_xlabel(r'$\hat{B_{44}}$ (model test)')
    #ax.set_ylabel(r'$\hat{B_{44}}$ (prediction)')
        
    ax.set_xlim(0,lim)
    ax.set_ylim(0,lim)
    #ax.set_title('Total roll damping for Ikeda, Simplified Ikeda and model tests')
    ax.plot([0,lim],[0,lim],'r-')
    
    ax.grid(True)
    ax.set_aspect('equal', 'box')
    ax.legend();